In [2]:
%load_ext autoreload
%autoreload 2
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import pandas as pd
import os
import sys
import numpy as np
sys.path.append("../../Code")
import utils
import files as f
import seaborn as sns
import matplotlib.pyplot as plt
import preprocessing as pre
import languages as lang
stopWords = set(stopwords.words('english'))
import profiles
import afinns as af

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [41]:
def end_review(text):
    terminals ="?!."
    if text[-1] not in terminals:
        return text + "."
    else:
        return text

# Load Guests, Hosts, Reviews , and reverse reviews 

In [58]:
guests = pd.read_csv(os.path.join('..','2','nr_restrict_guests.csv'), low_memory=False, index_col=0).drop_duplicates(subset="id", keep="last")
hosts = pd.read_csv(os.path.join('..','2','nr_restrict_hosts.csv'), index_col=0)
hosts = hosts[~hosts.index.isnull()]
reviews = pd.read_csv(os.path.join('..','2','nr_restrict_reviews.csv'), index_col=0)

In [26]:
host_reviews = pd.read_csv(os.path.join('..','2','man_hostReviews.csv'), index_col=0)
host_reviews = host_reviews[host_reviews['reviewer_id'].isin(hosts.id.unique())]

In [27]:
host_reviews['reviewer_id'] = host_reviews['reviewer_id'].astype(int).astype(str)

In [28]:
host_reviews = host_reviews[~host_reviews['comments'].isnull()]

In [29]:
len(guests)

96255

In [30]:
len(hosts)

6924

In [31]:
guest_reviews_corr = pd.read_csv(os.path.join('..','5','guest_reviews_spell.csv'), index_col=0)

In [32]:
host_reviews_corr = pd.read_csv(os.path.join('..','5','host_reviews_spell.csv'), index_col=0)

In [33]:
len(guest_reviews_corr)

225090

In [34]:
len(host_reviews_corr)

36202

## Get Personality from all outward reviews

In [35]:
len(guests.id.unique())

96255

In [36]:
len(guests.id)

96255

### Merge All Reviews First 

In [37]:
guests[guests.id == 178500510]

,idGuest,id,name,city,ccode,membershipMonth,membershipYear,superhost,verified,description,linkedAccountVerified,schoolInfo,jobInfo,languages,reviewNumber,guideNumber,wishListNumber


In [38]:
g_ids = set(guests['id']).union(set(guest_reviews_corr['reviewer_id']))
list_g_ids = list(g_ids)

In [39]:
guests_full = pd.DataFrame({'id': list_g_ids })

In [42]:
g_reviews = []
g_num_reviews = []
for g in list_g_ids:
    revs = [end_review(c) for c in guest_reviews_corr[guest_reviews_corr['reviewer_id'] == g]['comments']]
    g_reviews.append(revs)
    g_num_reviews.append(len(revs))  

In [43]:
guests_full['review_text'] = g_reviews
guests_full['num_reviews'] = g_num_reviews

In [44]:
guests_full.head()

,id,review_text,num_reviews
0,1,[Great spot. I enjoyed the location and the sp...,2
1,3,[Stayin at Asher's Guest Room in his Art Loft ...,1
2,40894469,[We had a wonderful stay in New York! Carlinho...,1
3,26214407,"[Our first airbnb experience was fantastic, al...",1
4,2097159,"[Thanks Adam, Kristine, and Seth! While the o...",1


In [45]:
h_ids = set(hosts['id']).union(set(host_reviews_corr['reviewer_id']))
list_h_ids = list(h_ids)

In [46]:
hosts_full = pd.DataFrame({'id': list_h_ids })

In [47]:
h_reviews = []
h_num_reviews = []
for h in list_h_ids :
    revs = [end_review(c) for c in host_reviews_corr[host_reviews_corr['reviewer_id'] == h]['comments']]
    h_reviews.append(revs)
    h_num_reviews.append(len(revs)) 

In [48]:
hosts_full['review_text'] = h_reviews
hosts_full['num_reviews'] = h_num_reviews

In [49]:
guests_full.to_csv('guests_outward_review_full.csv')
hosts_full.to_csv('hosts_outward_review_full.csv')

# Personality

In [50]:
guests = pd.read_csv('guests_outward_review_full.csv', index_col=0)

In [51]:
guest_profile = pd.read_csv('guest_profile_spell.csv', low_memory=False, index_col=0)

In [52]:
guest_text = guests.join(guest_profile[['id', 'description']].set_index('id'), on='id')

In [53]:
hosts = pd.read_csv('hosts_outward_review_full.csv', index_col=0)

In [54]:
host_profile = pd.read_csv('host_profile_spell.csv', low_memory=False, index_col=0)

In [55]:
host_text = hosts.join(host_profile[['id', 'about']].set_index('id'), on='id')

In [56]:
all_text = pd.concat([guest_text, host_text.rename(columns={'about':'description'})]).reset_index(drop=True)

In [57]:
all_text = all_text.drop_duplicates()

In [58]:
all_text = all_text[(all_text['review_text'] != '[]') |(~all_text['description'].isnull())]

In [59]:
n_reviews = []
merged_text = []
for _, r in all_text.iterrows():
    if (len(r['review_text']) == 0) and (r['description'].isnull()):
        n_reviews.append(r['num_reviews'])
        merged_text.append(None)
    elif (r['description'] in [np.nan, None]):
        n_reviews.append(r['num_reviews'])
        merged_text.append(r['review_text'])
    elif (r['review_text'] in [np.nan, None]):
        n_reviews.append(1)
        merged_text.append(r['description'])
    else:
        n_reviews.append(r['num_reviews'] + 1)
        merged_text.append(list([r['description']])+(eval(r['review_text']) ) )

In [60]:
all_text['num_of_texts'] = n_reviews
all_text['merged_texts'] = merged_text

In [61]:
all_text.to_csv('personality_text.csv')

In [62]:
all_text['personality'] = [lang.get_personality_metrics(t) for t in all_text['merged_texts']]

In [177]:
all_text.to_csv('personality.csv')

### Hosts

In [228]:
host_text = hosts_full.join(hosts[['id', 'about']].set_index('id'), on='id')

In [229]:
n_reviews = []
merged_text = []
for _, r in host_text.iterrows():
    if (len(r['review_text']) == 0) and (r['about'] in [np.nan, None]):
        n_reviews.append(r['num_reviews'])
        merged_text.append(None)
    elif (r['about'] in [np.nan, None]):
        n_reviews.append(r['num_reviews'])
        merged_text.append(r['review_text'])
    elif len(r['review_text']) == 0:
        n_reviews.append(1)
        merged_text.append(r['about'])
    else:
        n_reviews.append(r['num_reviews'] + 1)
        merged_text.append([r['about']]+r['review_text']) 

In [230]:
host_text['num_of_texts'] = n_reviews
host_text['merged_texts'] = merged_text

In [231]:
host_text.to_csv('hosts_outward_full.csv')

In [232]:
host_text['personality'] = [lang.get_personality_metrics(t) for t in host_text['merged_texts']]

In [233]:
host_text.to_csv('host_review_personality.csv')

In [203]:
pd.options.display.max_colwidth = 10000 # 7118

In [3]:
a = pd.read_csv('host_review_personality.csv', index_col=0)

In [4]:
a

,id,review_text,num_reviews,about,num_of_texts,merged_texts,personality
0,7118,"['The guests I met seemed cool and nice, and m...",21,Professional Designer/Artist working in Theatr...,22,"[""Professional Designer/Artist working in Thea...","{'avg_wc': 14.272727272727273, 'avg_word_len':..."
1,7322,[],0,"I'm an artist, writer, traveler, and a native ...",1,"I'm an artist, writer, traveler, and a native ...","{'avg_wc': 0.7662337662337663, 'avg_word_len':..."
2,7386,['Bill arrived and left at the times he said h...,44,wunderlustful itinerant smarty pants ... cooki...,45,['wunderlustful itinerant smarty pants ... coo...,"{'avg_wc': 21.555555555555557, 'avg_word_len':..."
3,8967,['It was a please sure to serve Janet here. Th...,49,I used to work for a financial industry but no...,50,['I used to work for a financial industry but ...,"{'avg_wc': 23.62, 'avg_word_len': 3.9974597798..."
4,7490,"['Bonnie is an outstanding guest: considerate,...",97,Welcome to family life with my oldest two away...,98,"[""Welcome to family life with my oldest two aw...","{'avg_wc': 37.60204081632653, 'avg_word_len': ..."
5,7702,['Fantastic guest! Isis was very easy to commu...,35,I am a New Yorker who loves to travel.,36,"['I am a New Yorker who loves to travel.', 'Fa...","{'avg_wc': 18.63888888888889, 'avg_word_len': ..."
6,7989,"['Stephen was a polite, quiet and respectful g...",112,I am an educator and researcher and I've worke...,113,"[""I am an educator and researcher and I've wor...","{'avg_wc': 41.16814159292036, 'avg_word_len': ..."
7,8526,"['Great guest !', 'Philippe and his family too...",32,I was born and raised in Arizona and have live...,33,['I was born and raised in Arizona and have li...,"{'avg_wc': 14.787878787878787, 'avg_word_len':..."
8,18583,"['great. nice, pleasant, easy-going.', 'i woul...",44,"Hey, I am John - Fun, nice, educated, clean, i...",45,"['Hey, I am John - Fun, nice, educated, clean,...","{'avg_wc': 29.0, 'avg_word_len': 4.31800766283..."
9,18946,['Fantastic! What a terrific family. Their jub...,118,Raised in Europe and Asia I love to host inter...,119,['Raised in Europe and Asia I love to host int...,"{'avg_wc': 52.705882352941174, 'avg_word_len':..."


## Classify values 

In [154]:
all_personality

,id,merged_texts,adjectives,adverbs,articles,avg_long_words,avg_sent_len,avg_wc,avg_word_len,capital,...,positive_words,pronouns,self,swear,tentative,to,unique_words,verbs,we,you
0,1,[I live with my wife who is a doctor at Stanfo...,0.076923,0.096154,0.057692,0.173077,31.777778,17.333333,4.442308,0.057692,...,0.134615,0.115385,0.000000,0.0,0.000000,0.019231,0.846154,0.192308,0.000000,0.000000
1,3,"[I am originally from NY, and have been living...",0.069149,0.079787,0.101064,0.186170,84.250000,94.000000,4.340426,0.047872,...,0.037234,0.074468,0.000000,0.0,0.021277,0.015957,0.686170,0.180851,0.005319,0.010638
2,40894469,['We had a wonderful stay in New York! Carlinh...,0.089385,0.000000,0.089385,0.000000,109.000000,0.806306,1.000000,0.039106,...,0.000000,0.039106,0.000000,0.0,0.000000,0.000000,0.128492,0.134078,0.000000,0.000000
3,26214407,[Living in the beautiful city of Winchester in...,0.108359,0.083591,0.083591,0.164087,116.733333,161.500000,4.343653,0.009288,...,0.055728,0.040248,0.003096,0.0,0.012384,0.030960,0.578947,0.185759,0.037152,0.000000
4,2097159,"[""Thanks Adam, Kristine, and Seth! While the ...",0.103077,0.000000,0.100000,0.000000,229.571429,0.795594,1.000000,0.027692,...,0.000000,0.027692,0.000000,0.0,0.000000,0.000000,0.041538,0.127692,0.000000,0.000000
5,2097168,"[""Matt was extremely proactive about making su...",0.123810,0.000000,0.093333,0.000000,165.250000,0.782414,1.000000,0.032381,...,0.000000,0.034286,0.000000,0.0,0.000000,0.000000,0.047619,0.116190,0.000000,0.000000
6,63963154,"[""A very nice apartment. Clean and comfortable...",0.116883,0.000000,0.142857,0.000000,48.000000,0.770000,1.000000,0.038961,...,0.000000,0.025974,0.000000,0.0,0.000000,0.000000,0.285714,0.116883,0.000000,0.000000
7,4194323,['Eddie’s place is fantastic - I would definit...,0.093333,0.000000,0.097143,0.000000,187.285714,0.790663,1.000000,0.019048,...,0.000000,0.013333,0.000000,0.0,0.000000,0.000000,0.045714,0.131429,0.000000,0.000000
8,63963158,[Professional. Like to embrace local culture w...,0.142857,0.040816,0.102041,0.163265,51.000000,24.500000,4.183673,0.000000,...,0.122449,0.040816,0.000000,0.0,0.000000,0.102041,0.795918,0.163265,0.000000,0.000000
9,156237850,['Thanks! great communication and terrific loc...,0.130952,0.011905,0.107143,0.000000,49.500000,0.815534,1.000000,0.023810,...,0.000000,0.023810,0.000000,0.0,0.000000,0.000000,0.250000,0.119048,0.000000,0.000000


In [152]:
personality_keys = ['adjectives', 'adverbs', 'articles', 'avg_long_words', 'avg_sent_len', 'avg_wc', 'avg_word_len', 'capital', 'conj', 'formality', 'i', 'interjections', 'modal', 'negations',\
                   'negative_words', 'nouns', 'other_pro', 'pos_count', 'positive_words',\
                   'pronouns', 'self', 'swear', 'tentative', 'to', 'unique_words', 'verbs',\
                   'we', 'you']

In [158]:
vals = {k:[] for k in personality_keys}
for i, r in all_text.iterrows():
    for k in personality_keys:
        vals[k].append(r['personality'][k] if r['personality'] not in [np.nan, None] else np.nan)
    

In [159]:
for v in vals.keys():
    all_text[v] = vals[v]

/Users/lynraybarends/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [153]:
all_personality = all_text[['id', 'merged_texts']].join(pd.DataFrame.from_records([{j:np.nan for j in personality_keys} if i == None else i for i in all_text['personality']]))




In [177]:
swear = []
for i, r in all_text.iterrows(): 
    text = str(" ".join(r['merged_texts']))
#     sent_token_text = sent_tokenize(text)
    token_text = utils.tokenize(text)
    swear.append(np.sum([1 for t in token_text if t.lower() in swear] if (len(token_text) == 0) else 0))

In [169]:
pd.options.display.max_colwidth = 500

In [176]:
swear = set(['fuck', 'shit', 'twat', 'bloody', 'cunt', 'bitch'])

In [180]:
all_text.to_csv("personality_raw.csv")

In [5]:
all_text = pd.read_csv('personality_raw.csv', index_col=0)

In [7]:
all_text[]

,id,num_reviews,num_of_texts,adjectives,adverbs,articles,avg_long_words,avg_sent_len,avg_wc,avg_word_len,...,pronouns,self,swear,tentative,to,unique_words,verbs,we,you,swear_words
count,2.168360e+05,216836.000000,216836.000000,216823.000000,216823.000000,216823.000000,216823.000000,216834.000000,216823.000000,216823.000000,...,216823.000000,216823.000000,216823.000000,216823.000000,216823.000000,216823.000000,216823.000000,216823.000000,216823.000000,216836.0
mean,4.916303e+07,1.205021,1.467916,0.099805,0.017415,0.093245,0.051070,93.976363,15.009251,1.913292,...,0.049608,0.000192,0.003759,0.002226,0.007594,0.308749,0.142429,0.002784,0.001459,0.0
std,4.733876e+07,3.161567,3.224241,0.027880,0.032079,0.026559,0.091196,48.434985,31.178844,1.542395,...,0.048618,0.001541,0.065420,0.006526,0.016211,0.282593,0.031719,0.009275,0.005643,0.0
min,1.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.571429,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001397,0.000000,0.000000,0.000000,0.0
25%,1.073555e+07,1.000000,1.000000,0.084567,0.000000,0.078652,0.000000,63.000000,0.784314,1.000000,...,0.023256,0.000000,0.000000,0.000000,0.000000,0.089849,0.123077,0.000000,0.000000,0.0
50%,3.269430e+07,1.000000,1.000000,0.099010,0.000000,0.093151,0.000000,85.777778,0.797101,1.000000,...,0.034014,0.000000,0.000000,0.000000,0.000000,0.167832,0.137143,0.000000,0.000000,0.0
75%,7.469016e+07,1.000000,2.000000,0.113565,0.017544,0.108108,0.111748,115.666667,12.500000,3.962963,...,0.057554,0.000000,0.000000,0.000000,0.000000,0.583333,0.157742,0.000000,0.000000,0.0
max,1.927242e+08,244.000000,245.000000,1.000000,1.000000,0.375000,1.000000,1930.000000,689.000000,12.000000,...,1.000000,0.142857,4.000000,0.200000,0.333333,1.000000,0.666667,0.181818,0.250000,0.0


In [41]:
all_text_stripped = all_text[['id','merged_texts', 'avg_sent_len', 'capital', 'formality', 'i', 'pronouns', 'unique_words', 'verbs']]

In [82]:
all_text_stripped[all_text_stripped.formality > 500]

,id,merged_texts,adverbs,avg_long_words,avg_sent_len,capital,formality,i,nouns,positive_words,pronouns,unique_words,verbs
43,10485919,"[""Carol and Marc are amazing hosts: they made ...",0.000609,0.000000,215.000000,0.035954,591.0,0.006094,0.619135,0.000000,0.034126,0.016453,0.127971
120,127926727,"[""It's the first time i use Airbnb and i reall...",0.001361,0.000000,228.500000,0.018367,552.0,0.007483,0.615646,0.000000,0.012925,0.018367,0.136054
227,27263879,['Eric’s place has many issues which are very ...,0.000456,0.000000,236.826087,0.011867,841.0,0.002282,0.656321,0.000000,0.010954,0.012780,0.123231
326,28312970,"[""Jernee and Seth were great hosts. The descri...",0.000000,0.000000,155.620690,0.035457,636.5,0.006648,0.614958,0.000000,0.030471,0.017175,0.132964
1109,30414039,"[""Allison was flexible in terms of arrival and...",0.000000,0.000000,133.000000,0.021973,797.0,0.002805,0.625526,0.000000,0.021505,0.014493,0.120617
1129,35657019,"[""Lexi's apartment was just as described - cha...",0.000705,0.000000,177.500000,0.028894,545.5,0.002114,0.631431,0.000000,0.027484,0.023961,0.112755
1675,57679769,"[""The charming East Village Studio was exactly...",0.001455,0.000000,148.565217,0.032727,508.0,0.010909,0.611636,0.000000,0.024000,0.018182,0.131636
1716,160440412,['Great size apartment. Perfect location for w...,0.000000,0.000000,121.518519,0.018854,526.0,0.002262,0.664404,0.000000,0.018854,0.025641,0.113122
1899,34612187,['The apartment itself was basically good and ...,0.000000,0.000000,187.500000,0.032240,657.5,0.011673,0.629794,0.000000,0.023902,0.016120,0.127849
1961,139470073,['LISTING DESCRIPTION The title of the listing...,0.001342,0.000000,216.294118,0.045638,542.0,0.010067,0.627517,0.000000,0.034228,0.016779,0.124832


In [83]:
"".join(all_text_stripped[all_text_stripped.id == 13969061]['merged_texts'])

'[\'Sara and her mom stayed in my apartment for 3 nights. They arrived early in the morning and communicated well regarding their arrival/departure time. They were friendly when they arrived and left the apartment in great condition. I highly recommend Sara for any future requests and welcome her to come back.\', \'Yael stayed at my apartment for 10 nights. The apartment was left in great condition. He communicated well in regards to the check-in/check-out process. Happy to host him and would welcome him to stay again. Highly recommend for any future stays.\', "Joseph and his girlfriend stayed 3 nights in my apartment. Though I never met him face to face, he was a good communicator and let me know exactly when he was planning to check in and check out. The apartment was left in good condition and did not require any extra cleaning. I\'d recommend Joseph for any future airbnb visits and would welcome him back to stay.", \'Shannen and her boyfriend stayed at my apartment for 3 nights. Th

In [79]:
"".join(all_text_stripped[all_text_stripped.id == 178295576]['merged_texts'])

"['THIS PLACE WAS GREAT FOR IT 2 BE MY FIRST TIME I HAD RE TIME OF MY LIFE... IT WAS VERY CLEAN AND SPACIOUS I WAS ABL TO COOK AND SHOWER COMFORTABLY..... THE HOST WAS VERY RESPECTFUL AND UNDERSTANDING WITH ME..... I LOVE DENIS HOME AS IF IT WAS MY OWN .... NEXT TIME I USED THIS APP DENIS WILL BE THE FIRST HOUSE I HOPE POP UP AND I RESERVE AGAIN.']"

In [61]:
reviews.index

Int64Index([   203,    204,    205,    206,    207,    208,    209,    210,
               211,    212,
            ...
            428579, 428918, 429333, 429543, 430047, 430814, 431090, 431475,
            431702, 432140],
           dtype='int64', length=215960)

In [63]:
reviews = reviews.join(all_text_stripped[['id', 'formality']].set_index('id'), on='reviewer_id').rename(columns={'formality': 'guest_formality'})

In [64]:
reviews = reviews.join(all_text_stripped[['id', 'formality']].set_index('id'), on='recipient_id').rename(columns={'formality': 'host_formality'})

In [70]:
reviews['formality_difference'] = reviews['host_formality'] - reviews['guest_formality']

In [73]:
reviews[['id', 'formality_difference', 'formality_difference_abs']].to_csv('formality_diff.csv')

In [56]:
all_text_stripped

,id,merged_texts,adverbs,avg_long_words,avg_sent_len,capital,formality,i,nouns,positive_words,pronouns,unique_words,verbs
0,1,['I live with my wife who is a doctor at Stanf...,0.096154,0.173077,31.777778,0.057692,50.5,0.057692,0.230769,0.134615,0.115385,0.846154,0.192308
1,3,"['I am originally from NY, and have been livin...",0.079787,0.186170,84.250000,0.047872,63.0,0.037234,0.175532,0.037234,0.074468,0.686170,0.180851
2,40894469,['We had a wonderful stay in New York! Carlinh...,0.000000,0.000000,109.000000,0.039106,107.0,0.000000,0.625698,0.000000,0.039106,0.128492,0.134078
3,26214407,['Living in the beautiful city of Winchester i...,0.083591,0.164087,116.733333,0.009288,73.0,0.003096,0.167183,0.055728,0.040248,0.578947,0.185759
4,2097159,"[""Thanks Adam, Kristine, and Seth! While the ...",0.000000,0.000000,229.571429,0.027692,272.0,0.000000,0.601538,0.000000,0.027692,0.041538,0.127692
5,2097168,"[""Matt was extremely proactive about making su...",0.000000,0.000000,165.250000,0.032381,230.5,0.001905,0.598095,0.000000,0.034286,0.047619,0.116190
6,63963154,"[""A very nice apartment. Clean and comfortable...",0.000000,0.000000,48.000000,0.038961,76.5,0.000000,0.558442,0.000000,0.025974,0.285714,0.116883
7,4194323,['Eddie’s place is fantastic - I would definit...,0.000000,0.000000,187.285714,0.019048,231.5,0.005714,0.632381,0.000000,0.013333,0.045714,0.131429
8,63963158,['Professional. Like to embrace local culture ...,0.040816,0.163265,51.000000,0.000000,57.5,0.000000,0.183673,0.122449,0.040816,0.795918,0.163265
9,156237850,['Thanks! great communication and terrific loc...,0.011905,0.000000,49.500000,0.023810,78.0,0.000000,0.547619,0.000000,0.023810,0.250000,0.119048


In [5]:
def classify(x, m, s):
    upper_thresh = m+s
    lower_thresh = m-s
    if x >= upper_thresh:
        return 'H'
    elif x > m:
        return 'h'
    elif x <= lower_thresh:
        return 'L'
    elif x < m:
        return "l"
    else:
        return 'm'
    

In [6]:
def classify_dimensions(tbl):
    result = pd.DataFrame({"id":tbl['id']})
    
    for c in tbl.drop('id', axis=1).columns :
        classes = []
        mean = np.mean(tbl[c])
        stddev = np.std(tbl[c])
        classes = tbl[c].map(lambda x: classify(x, mean, stddev))
        result[c] = classes
    return result

In [7]:
class_personality = classify_dimensions(all_text_stripped)

KeyboardInterrupt: 

In [21]:
personalities = {
    'neurotism': {
        'i': 'h',
        'neg_words': 'h',
        'pos_words': 'l',
        'articles': 'l',
        'we': 'l',
        'negations': 'h',
        'self': 'h',
        'preps': 'l'
    },
    'extraversion': {
        'positive_words': 'h',
        'i': 'h',
        #High number of verbs,adverbs,pronouns,
        'negative_words': 'l',
        'negations': 'l',
        'avg_sent_len': 'l',
        'articles': 'l',
        'unique_words': 'l',
        'pos_count': 'l',
        'formality': 'l',
        'we': 'h',
        'self': 'h'
        
    },
    'openness': {
        'unique_words': 'h',
        'avg_long_words': 'h',
        'tentative': 'h', 
        'i': 'l',
        #low number of present tense words,
        'articles': 'h',
        'you': 'h',
        'swear':'h',
        'self': 'l',
        'preps': 'l'
    },
    'agreeableness': {
        'positive_words': 'h',
        'negative_words': 'l',
        'negations': 'l',
        'i': 'h',
        'articles': 'h',
        'we': 'h',
        'you': 'l',
        'swear':'l',
        'self': 'h'
    },
    'conscientiousness': {
        'articles': 'l',
        'negative_words': 'l',
        'negations': 'l',
        'tentative': 'h',
        'unique_words': 'h',
        'positive_words': 'h',
        'you': 'l',
        'swear':'l',
        'self': 'h',
        'preps': 'h'
    }
}

In [191]:
def get_personality(tbl, personality):
    result = []
    for i,r in tbl.iterrows():
        scores = []
        for k in personality.keys():
            if r[k].lower() == personality[k]:
                val = 1
            elif r[k].upper() == personality[k]:
                val = 2
            else: 
                val =0
            scores.append(val)
        result.append(np.mean(scores))
    return result
    

In [22]:
def get_classifications(tbl, personality):
    classifications = pd.DataFrame()
    for f in personality.keys():
        mean  = np.average(tbl[f])
        stddev = np.std(tbl[f])
        upper_thresh = mean+stddev
        lower_thresh = mean-stddev
        vals = []
        if personality[f] is 'h' :
            vals = tbl[f].map(lambda x: 2 if x  > upper_thresh  else 0)
        else :
            vals = tbl[f].map(lambda x: 2 if x  < lower_thresh  else 0)
        classifications[f] = vals
    return np.average(classifications, axis=1)
      

In [24]:
profile_person.columns

Index(['id', 'sent_count', 'qmarks', 'avg_nums', 'conj', 'brackets', 'self',
       'foreign', 'avg_word_len', 'other_pro', 'pos_count', 'formality',
       'prepositions', 'unique_words', 'articles', 'capital', 'exclaim', 'we',
       'when', 'adverbs', 'negations', 'wc', 'adjectives', 'you', 'particles',
       'i', 'verbs', 'interjections', 'modal', 'pronouns', 'avg_sent_len',
       'stop', 'nouns', 'avg_long_words'],
      dtype='object')

In [23]:
get_classifications(profile_person, personalities['neurotism'] )

KeyError: 'neg_words'

In [192]:
g_big_five = pd.DataFrame({'id': class_g_personality.id})

In [193]:
g_big_five['neurotism'] = get_personality(class_g_personality, personalities['neurotism'])
g_big_five['extraversion'] = get_personality(class_g_personality, personalities['extraversion'])
g_big_five['openness'] = get_personality(class_g_personality, personalities['openness'])
g_big_five['conscientiousness'] = get_personality(class_g_personality, personalities['conscientiousness'])
g_big_five['agreeableness'] = get_personality(class_g_personality, personalities['agreeableness'])

In [237]:
h_big_five = pd.DataFrame({'id': class_h_personality.id})

In [238]:
h_big_five['neurotism'] = get_personality(class_h_personality, personalities['neurotism'])
h_big_five['extraversion'] = get_personality(class_h_personality, personalities['extraversion'])
h_big_five['openness'] = get_personality(class_h_personality, personalities['openness'])
h_big_five['conscientiousness'] = get_personality(class_h_personality, personalities['conscientiousness'])
h_big_five['agreeableness'] = get_personality(class_h_personality, personalities['agreeableness'])

In [263]:
h_big_five['id'] = h_big_five['id'].astype(int)

## Submitting values 

In [134]:
h_personality['id'] = h_personality['id'].astype(int)

In [240]:
g_reviews = reviews[['id', 'reviewer_id']].join(g_big_five.set_index('id'), on ='reviewer_id')

In [264]:
h_reviews = reviews[['id', 'recipient_id']].join(h_big_five.set_index('id'), on ='recipient_id')

In [242]:
cols = set(h_reviews.columns) - set(['id', 'recipient_id', 'reviewer_id'])

In [266]:
result = pd.DataFrame({'id': list(reviews.id)})

In [267]:
len(result)

215960

In [272]:
result[~result.neurotism.isnull()]

,id,agreeableness,extraversion,neurotism,openness,conscientiousness
203,401808,0.2,0.1,0.00,0.2,0.0
204,425216,0.2,0.2,0.00,0.4,0.0
205,454928,0.2,0.0,0.00,0.0,0.2
206,475719,0.2,0.2,0.25,0.0,0.4
207,540860,0.0,0.4,0.00,0.2,0.4
208,582546,0.2,0.1,0.25,0.6,0.4
209,630702,0.0,0.2,0.25,0.0,0.2
210,644485,0.2,0.0,0.25,0.4,0.2
211,686637,0.4,0.0,0.25,0.2,0.0
213,740991,0.6,0.2,0.50,0.2,0.2


In [271]:
for c in cols: 
    result.is_copy = False
    result[str(c)] = np.abs(h_reviews[str(c)] - g_reviews[str(c)])

In [274]:
result[~result.neurotism.isnull()].to_csv('reviews_linguistic.csv')

In [3]:
profile_person = pd.read_csv('profile_linguistic.csv', index_col=0)

In [4]:
len(profile_person)

46969

In [5]:
profile_person

,id,sent_count,qmarks,avg_nums,conj,brackets,self,foreign,avg_word_len,other_pro,...,particles,i,verbs,interjections,modal,pronouns,avg_sent_len,stop,nouns,avg_long_words
203,401808,0.0,0.0,0.000000,-0.107843,0.0,0.000000,0.0,0.049020,0.000000,...,0.000000,-0.166667,-0.274510,0.0,0.000000,0.294118,80.000000,1.0,-0.098039,-0.029412
206,475719,-1.0,0.0,0.000000,-0.095023,0.0,0.000000,0.0,2.113122,0.000000,...,0.000000,-0.153846,-0.248869,0.0,0.000000,0.294118,89.000000,-1.0,0.158371,0.393665
207,540860,0.0,0.0,-0.086957,-0.028133,0.0,0.000000,0.0,2.012788,-0.086957,...,0.000000,-0.043478,0.015345,0.0,0.000000,0.207161,9.000000,0.0,0.017903,0.383632
208,582546,-8.0,0.0,-0.031496,0.003705,0.0,-0.007874,0.0,1.354794,-0.015748,...,0.000000,-0.031496,-0.114405,0.0,0.000000,0.191755,42.666667,-6.0,0.085688,0.281612
209,630702,0.0,0.0,0.000000,0.058824,0.0,0.000000,0.0,0.082353,0.000000,...,0.000000,-0.200000,-0.141176,0.0,0.000000,0.294118,88.000000,1.0,-0.364706,0.070588
210,644485,-6.0,0.0,-0.037037,0.021786,-1.0,0.000000,0.0,2.339143,0.000000,...,0.000000,-0.074074,-0.126362,0.0,-0.024691,0.195352,67.428571,-9.0,0.050109,0.408860
211,686637,-1.0,0.0,0.000000,0.058824,0.0,0.000000,0.0,1.310924,0.000000,...,0.000000,0.000000,-0.084034,0.0,0.000000,-0.134454,101.500000,-1.0,0.235294,0.184874
213,740991,-7.0,0.0,-0.021505,0.005060,0.0,-0.010753,0.0,1.161923,-0.010753,...,0.000000,-0.021505,-0.070209,0.0,-0.021505,0.122075,53.500000,-7.0,0.030993,0.244782
214,781890,0.0,0.0,0.000000,0.006192,0.0,0.000000,0.0,1.198142,0.000000,...,0.000000,-0.052632,-0.099071,0.0,0.000000,0.083591,11.000000,0.0,0.130031,0.260062
217,852090,-2.0,0.0,-0.096774,0.026565,0.0,0.000000,0.0,1.946869,0.000000,...,0.000000,-0.096774,-0.166983,0.0,0.000000,0.100569,68.333333,-1.0,0.170778,0.309298


In [218]:
from watson_developer_cloud import WatsonApiException, PersonalityInsightsV3
import json 
try:
    # Invoke a Personality Insights method
    personality_insights = PersonalityInsightsV3(
        version='2017-10-13',
        username='a6897f4c-354d-4c0d-a998-1ffc367118b9',
        password='SPGpduAwCf24',
        url='https://gateway.watsonplatform.net/personality-insights/api'
    )
    
    with open('profile.txt') as personality_text:
        print(json.dumps(personality_insights.profile(content_type='text/plain', content=personality_text.read() ), indent=2))


except WatsonApiException as ex:
    print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

{
  "word_count": 314,
  "word_count_message": "There were 314 words in the input. We need a minimum of 600, preferably 1,200 or more, to compute statistically significant estimates",
  "processed_language": "en",
  "personality": [
    {
      "trait_id": "big5_openness",
      "name": "Openness",
      "category": "personality",
      "percentile": 0.9148457167873705,
      "significant": true,
      "children": [
        {
          "trait_id": "facet_adventurousness",
          "name": "Adventurousness",
          "category": "personality",
          "percentile": 0.04811042796781162,
          "significant": true
        },
        {
          "trait_id": "facet_artistic_interests",
          "name": "Artistic interests",
          "category": "personality",
          "percentile": 0.7356618254848829,
          "significant": true
        },
        {
          "trait_id": "facet_emotionality",
          "name": "Emotionality",
          "category": "personality",
          "perce